Let's first import necessary libraries

In [1]:
# Imported for proper rendering of latex in colab.
from sklearn.model_selection import train_test_split
from IPython.display import display, Math, Latex
import numpy as np

# Import for generating plots
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### N4 : Optimization
The objective of this notebook is to implement optimization component of linear regression model.

It is implemented with one of the following two methods:
* Normal equation method : Sets the partial derivative of the loss function w.r.t. weight vector to 0 and solves the resulting equation to obtain the weight vector.

* Gradient descent method : Iteratively adjusts the weight vector based on the learning rate and the gradient of loss function at the current weight vector.


#### 1. Normal equation method

In [2]:
def normal_equation(X, y):
    ''' Estimates parameters of the linear regression model with normal eqn.
        
        Args:
            X: Feature matrix for given inputs.
            y: Actual label vector.
        Returns:
            weight vector
    '''

    return np.linalg.pinv(X) @ y

We test this function with the generated training set whose vector is known to use. 
* We set up the test with feature matrix, label vector and the expected weight vectors.

* Next we estimate the weight vector with ```normal_equation``` function.

* We test(a) shape and (b) match between expected and estimated weight vectors.

In [3]:
w1 = 5 
w0 = 6 
n = 100
X = np.random.rand(n,)
y = w0+w1*X+np.random.randn(n,)

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.20, random_state=42)

In [4]:
import unittest 
class TestNormalEquation(unittest.TestCase):

    def test_normal_equation(self):

        ''' Test case for weight estimation for linear regression with normal equation method.'''

        #setup
        feature_matrix =  X_train
        label_vector = y_train 
        expected_weight_vector = np.array([5.,6.])

        #call
        estimated_weight_vector = normal_equation(feature_matrix, label_vector)

        # asserts
        # test the shape
        self.assertEqual(estimated_weight_vector.shape, (2, ))
        #and contents
        np.testing.assert_array_almost_equal(estimated_weight_vector , expected_weight_vector , decimal=0)
      
unittest.main(argv=[''],defaultTest='TestNormalEquation',verbosity=2,exit=False)      

test_normal_equation (__main__.TestNormalEquation)
Test case for weight estimation for linear regression with normal equation method. ... ERROR

ERROR: test_normal_equation (__main__.TestNormalEquation)
Test case for weight estimation for linear regression with normal equation method.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\faizan\AppData\Local\Temp/ipykernel_31772/1352503705.py", line 14, in test_normal_equation
    estimated_weight_vector = normal_equation(feature_matrix, label_vector)
  File "C:\Users\faizan\AppData\Local\Temp/ipykernel_31772/1977596150.py", line 11, in normal_equation
    return np.linalg.pinv(X) @ y
  File "<__array_function__ internals>", line 5, in pinv
  File "c:\Users\faizan\anaconda3\envs\tensorflow\lib\site-packages\numpy\linalg\linalg.py", line 2002, in pinv
    u, s, vt = svd(a, full_matrices=False, hermitian=hermitian)
  File "<__array_function__ internals>", line 5, in svd

### 2. Gradient descent (GD):

GD is implemented as follows:
* Randomly initialize $\textbf w$ to $\textbf 0$.

* Iterate until convergence:
    * Calculate partial derivative of loss w.r.t weight vector.

    * Calculate new values of weights.

    * Update weights to new values $\textit {simultaneously}$. 
    
We use number of epochs as a convergence criteria in this implementation.

#### *Partial derivative of loss function :*

Let's first implement a function to calculate partial derivative of loss funciton, which is obtained with the following equation.
\begin{equation} 
\frac{∂}{∂ \textbf w}  J(\textbf w)= \textbf X^T(\textbf{Xw}-\textbf y)
\end{equation} 

The multiplication of transpose of feature matrix with the difference of predicted and actual label  vectors.


In [5]:
def predict(X, w):
    assert X.shape[-1] == w.shape[0], "X and w don't have compatible dimensions"
    return X @ w

In [6]:
def calculate_gradient(X,y,w):
    
    ''' Calculates gradients of loss function w.r.t weight vector on training set.
        Arguments:
            X: Features matrix for training data.
            y: Label vector for training data.
            w: Weight vector
        Retruns:
            A vector of gradients.
    '''
    return np.transpose(X)@(predict(X,w)-y)

Let's write  a test case for gradient calculation wtih the following setup:
1. Feature matrix $(\mathbf X)$
\begin{equation} 
\textbf X_{2 \times 4} = \begin{bmatrix} 1&3&2&5\\ 
1&9&4&7\end{bmatrix} 
\end{equation}
2. weight vector $(\textbf w)$
\begin{equation} \textbf w_{4\times 1}= \begin{bmatrix}1\\1\\1\\1 \end{bmatrix} 
\end{equation}
3. Label Vector $(y)$: 
\begin{equation} y_{2 \times 1} =  \begin{bmatrix} 6\\11   \end{bmatrix} 
\end{equation}


Let's compute the partial derivative of loss $ J(\mathbf w) $ i.e
\begin{align} \frac{∂}{∂ 
\textbf w}  J(\textbf w)&=& \textbf X^T(\textbf{Xw}-\textbf y) 
\end{align}

\begin{align}
\\ &=&\begin{bmatrix} 15\\105\\50\\95 \end{bmatrix}
\end{align}

In [7]:
class TestCalculateGradient(unittest.TestCase):
    def test_calculate_gradient(self):
        ''' Test case for gradient calculation. '''
        #set up
        feature_matrix = np.array([[1, 3, 2, 5], [1, 9, 4, 7]])
        weight_vector = np.array([1, 1, 1, 1])
        label_vector = np.array([6, 11])
        expected_grad = np.array([15, 105, 50, 95])

        #call
        grad = calculate_gradient(feature_matrix, label_vector, weight_vector)

        #asserts
        #test the shape
        self.assertEqual(grad.shape, (4, ))
        #and contents
        np.testing.assert_array_almost_equal(expected_grad, grad, decimal=0)


unittest.main(argv=[''], defaultTest='TestCalculateGradient', verbosity=2, exit=False)


test_calculate_gradient (__main__.TestCalculateGradient)
Test case for gradient calculation. ... ok

----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


#### Next Step : *Weight Updation*
We obtain the new weight from the old one by substracting gradient weighted by the learning rate.


In [8]:
def update_weights(w, grad, lr):
    ''' Updates the weights based on the gradient of loss function.

        Args:
            1. w: weight vector
            2. grad: gradient of loss wrt w
            3. lr: learning rate (alpha)
        Returns:
            Updated weight vector
    '''
    return (w - lr*grad)


Let's workout a weight update for :
1. Weight vector $(\textbf w)$: 
\begin{equation} 
\textbf w_{4\times 1}=\begin{bmatrix} 1\\1\\1\\1 \end{bmatrix} 
\end{equation} 
2. grad vector $(\textbf g)$: 
\begin{equation} 
\textbf g_{4\times 1}=\begin{bmatrix} 15\\105\\50\\95 \end{bmatrix} 
\end{equation}
3. Learning rate = 10^-3

The updated weights are given by:

\begin{align} 
\textbf w&=&\textbf w_{old} -\alpha \mathbf g 
\end{align} 

\begin{align} 
\\&=& 
\begin{bmatrix} 
1\\1\\1\\1\\ \end{bmatrix} -0.001\times 
\begin{bmatrix} 
15\\105\\50\\95 &
\end{bmatrix}
\end{align} 

\begin{align} 
\\ &=& \begin{bmatrix} 1-0.015\\1-0.105\\1-0.05\\1-0.095 \end{bmatrix} 
\end{align} 

\begin{align} 
\\&=& \begin{bmatrix} 0.985\\0.895\\0.95\\0.095 
\end{bmatrix} 
\end{align} 



In [9]:
class TestUpdateWeights(unittest.TestCase):
    def test_update_weights(self):
        ''' Test case for weight update in GD'''
        #set up
        weight_vector = np.array([1, 1, 1, 1])
        grad_vector = np.array([15, 105, 50, 95])
        lr = 0.001
        expected_w_new = np.array([0.985, 0.895, 0.95, 0.905])

        #call
        w_new = update_weights(weight_vector, grad_vector, lr)

        #asserts
        #test the shape
        self.assertEqual(expected_w_new.shape, (4,))

        #and contents
        np.testing.assert_array_almost_equal(expected_w_new, w_new, decimal=1)


unittest.main(argv=[''], defaultTest='TestUpdateWeights',
              verbosity=2, exit=False)


test_update_weights (__main__.TestUpdateWeights)
Test case for weight update in GD ... ok

----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


#### Next Step : *Gradient Descent*

In [10]:
def loss(X, y, w):
    e = (predict(X, w)) - y
    return (1/2)*((np.transpose(e))@e)

In [11]:
def gradient_descent(X: np.ndarray, y: np.ndarray, lr: float, num_epochs: int):

    w_all = []
    err_all = []
    w = np.zeros((X.shape[1]))
    print()
    for i in np.arange(0, num_epochs):
        w_all.append(w)
        err_all.append(loss(X, y, w))
        dJdW = calculate_gradient(X, y, w)

        if (i % 100) == 0:
            print('Iteration #:%d,loss: %4.2f' % (i, err_all[-1]))
        w = update_weights(w, dJdW, lr)
    return w, err_all, w_all


In order to test this function, we will use the synthetic training data that was generated earlier. We know the acutal weigths, that can be compared against the weights obtained from gradient descent procedure.

In [12]:
w1 = 5 
w0 = 6 
n = 100
X = np.random.rand(n,)
y = w0+w1*X+np.random.randn(n,)

X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [13]:
class TestGradientDescent(unittest.TestCase):
    def test_gradient_descent(self):
        ''' Test case for weight update in GD'''

        ''' Test case for gradient calculation. '''
        #set up
        feature_matrix = X_train
        label_vector = y_train
        expected_weight = np.array([5., 6.])

        #call
        w, err_all, w_all = gradient_descent(feature_matrix, label_vector, lr=0.001, num_epochs=2000)

        #asserts
        #test the shape
        self.assertEqual(w.shape, (2, ))
        #and contents
        np.testing.assert_array_almost_equal(expected_weight, w, decimal=0)


unittest.main(argv=[''], defaultTest='TestGradientDescent',verbosity=2, exit=False)


test_gradient_descent (__main__.TestGradientDescent)
Test case for weight update in GD ... ERROR

ERROR: test_gradient_descent (__main__.TestGradientDescent)
Test case for weight update in GD
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\faizan\AppData\Local\Temp/ipykernel_31772/4092288632.py", line 12, in test_gradient_descent
    w, err_all, w_all = gradient_descent(
  File "C:\Users\faizan\AppData\Local\Temp/ipykernel_31772/3006440603.py", line 5, in gradient_descent
    w = np.zeros((X.shape[1]))
IndexError: tuple index out of range

----------------------------------------------------------------------
Ran 1 test in 0.002s

FAILED (errors=1)


#### *Mini Batch Gradient Descent*

The key idea here to perform weight updates by computing gradient on batches of small number of examples.


In [14]:
t0, t1 = 200, 100000

def learning_schedule(t):
    return t0/(t+t1)


In [15]:
def mini_batch_gd(X: np.ndarray, y: np.ndarray, num_iters: int, minibatch_size: int):
    ''' Estimates parameters of linear regression model through gradient descent.

        Args: 
        X: feature matrix for training data
        y: label vector  for training data
        num_iters: number of iterations.

        Returns:
        weight vector: Final weight vector
        Error vector across different iterations
        weight vectors across different iterations
    '''

    w_all = []  # all parameters across iterations.
    err_all = []  # error across iterations

    #parameter vector initialized to [0,0]
    w = np.zeros((X.shape[1]))

    t = 0
    for epoch in range(num_iters):

        shuffled_indices = np.random.permutation(X.shape[0])
        X_shuffled = X[shuffled_indices]
        y_shuffled = y[shuffled_indices]

        for i in range(0, X.shape[0], minibatch_size):
            t += 1
            xi = X_shuffled[i, i + minibatch_size]

            yi = y_shuffled[i, i + minibatch_size]

            err_all.append(loss(xi, yi, w))

            gradients = 2/minibatch_size * calculate_gradient(xi, yi, w)
            lr = learning_schedule(t)

            w = update_weights(w, gradients, lr)
            w_all.append(w)
    return w, err_all, w_all


In [16]:
def test_minibatch_gd(self):

    #set up 
    feature_matrix = X_train 
    label_vector = y_train 
    expected_weights = np.array([4.,3.])


    #call 
    w, err_all, w_all = mini_batch_gd(feature_matrix , label_vector ,200 , 8)

    #asserts 
    #test the shape 
    self.assertEqual(w.shape ,(2,))

    #and contents 
    np.testing.assert_array_almost_equal(expected_weights, w, decimal=0)

unittest.main(argv=[''],defaultTest='TestMiniBatchGradientDescent',verbosity=2,exit=False)

TestMiniBatchGradientDescent (unittest.loader._FailedTest) ... ERROR

ERROR: TestMiniBatchGradientDescent (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute 'TestMiniBatchGradientDescent'

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (errors=1)
